In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("..")
sys.path.append("../upload_subjects/")

In [13]:
import argparse, os, cv2, re
import utils.db_utils as db_utils
import pandas as pd
import numpy as np
import pims

from PIL import Image
from datetime import date
from utils.zooniverse_utils import auth_session
from panoptes_client import (
    SubjectSet,
    Subject,
    Project,
    Panoptes,
)
from upload_frames import get_species_frames, extract_frames
from utils.summary_utils import clips_summary

In [14]:
db_path = "/data/db_config/koster_lab-nm.db"
user = "jannesg"
password = "Innolounge123"
conn = db_utils.create_connection(db_path)
frames_folder = "/data/deepsea_anemone_frames/"

In [15]:
clips_summary(db_path)

,species_id,how_many
label,,
Bivalves (any species),23.0,1.0
Black brittle star,15.0,51.0
Common sea pen,6.0,9.0
Common sea urchin,18.0,21.0
Common sunstar,16.0,1.0
Coral (any species),9.0,12.5
Crustacean (any species),14.0,19.0
Cushin star,17.0,15.0
Deep sea king crab,12.0,47.0


In [16]:
#### Connect to koster_db
conn = db_utils.create_connection(db_path)

# Connect to Zooniverse
koster_project = auth_session(user, password)

# Get id of species of interest
species_id = 5.0

# Identify n number of frames per classified clip that contains species of interest 
sp_frames_df = get_species_frames(species_id, conn, 3)

# Get info of frames already uploaded
uploaded_frames_df = pd.read_sql_query(
    f"SELECT movie_id, frame_number, frame_exp_sp_id FROM subjects WHERE frame_exp_sp_id='{species_id}' and subject_type='frame'",
    conn,
)

# Upload frames to Zooniverse that have not been uploaded
if len(sp_frames_df) == 0:
    print(
        "There are no subjects to upload, this may be because all of the subjects have already been uploaded"
    )
    raise

else:
    # Create the folder to store the frames if not exist
    if not os.path.exists(frames_folder):
        os.mkdir(frames_folder)

    # Extract the frames and save them
    sp_frames_df["frame_path"] = extract_frames(sp_frames_df, frames_folder)

    # Select koster db metadata associated with each frame
    sp_frames_df["label"] = species_id
    sp_frames_df["subject_type"] = "frame"

    sp_frames_df = sp_frames_df[
        [
            "frame_path",
            "fpath",
            "frame_number",
            "fps",
            "movie_id",
            "label",
            "frame_exp_sp_id",
            "subject_type",
        ]
    ]

    # Save the df as the subject metadata
    subject_metadata = sp_frames_df.set_index('frame_path').to_dict('index')

Videos are read in
1632
frames added to df
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_4500_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_4525_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_4550_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_0_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_25_5.0.jpg


/usr/src/app/koster_lab_development/src/pims/pims/pyav_reader.py:232: MethodDeprecationWarning: VideoStream.seek is deprecated.
  self._stream.seek(timestamp + self._first_pts)


saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_50_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_18000_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_18025_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_18050_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_36012_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_36037_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_36062_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_49512_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_49537_5.0.jpg
saving /data/deepsea_anemone_frames/000114 TMBL-ROV 2000 Säckenrevet Tape 55_frame_49562_5.0.jpg
saving /data/deepsea_anemone_fram